# Introduction
After the previous notebook (47) we have gone through all of the dead end metabolites and check wether or not they should be included in the biomass reaction. After doing so, we need to check that we get maximum of 1 g/gCDW when adding the stoichiometreisa and the weights of each compound together. Here I will check that.

If our biomass reaction exceeds the 1g/1gCDW, we need to scale all stoichiometries down to max 1 g/gCDW by changing the assigned stoichiometries linearly. This notebook will cover that.

Then when that has been finished, we can fit the GAM to the reaction to assure this fits measured chemostat data.


In [14]:
import cameo
import pandas as pd
import cobra.io
import escher
from escher import Builder
from cobra import Metabolite

In [33]:
model = cobra.io.read_sbml_model('../../model/g-thermo.xml')

In [34]:
#check total g/gdcw biomass
biomass_g_g = []
for met in model.reactions.biomass.metabolites:
    stoich = model.reactions.biomass.metabolites.get(met) #get stoichiometry in mmol/gcdw
    if stoich == 1 :#ignore the biomass part, but need to include the ADP/Pi/UDP that is also made here
        continue
    else:
        mw = met.formula_weight # get molecular weight of metabolite in g/mol
        try:
            weight_fraction = (mw/1000)*stoich #the g/gcdw of this metabolite
            biomass_g_g.append(weight_fraction)
        except TypeError: #for the met with R in it, just ignore
            print(met.id)

In [35]:
#to find total g/gcdw of all mets, sum up this list
abs(sum(biomass_g_g))

1.1169736189545567

So we need to scale the coefficients of all molecules in the biomass reaction to make sure this number is around 1 g/gCDW. 

To do so, we need to decrease the total stoichiometries of the metabolites. It needs to be scaled in such a way that the %of each metabolite in a g/g way remains the same compared to the rest. So, I need to scale based on the g/g of each metabolite.

In [36]:
for met in model.reactions.biomass.metabolites:
    if met.id in ['atp_c','adp_c','pi_c','h_c', 'Biomass_c', 'h2o_c']: #don't scale the biomass or ATP related metabolites. these will be fit in the GAM and they shouldn't contibute much to total weight anyway
        continue
    else:
        stoich = model.reactions.biomass.metabolites[met]
        new_stoich = stoich/1.12
        add_stoich = (stoich-new_stoich)
        model.reactions.biomass.add_metabolites({model.metabolites.get_by_id(met.id): -add_stoich})

We can see that the stoichiometry of the metabolites have changed accordingly. Lets check we now have 1g/gCDW.

In [37]:
#check total g/gdcw biomass
biomass_g_g_new = []
for met in model.reactions.biomass.metabolites:
    stoich = model.reactions.biomass.metabolites.get(met) #get stoichiometry in mmol/gcdw
    if stoich == 1 :#ignore the biomass part, but need to include the ADP/Pi/UDP that is also made here
        continue
    else:
        mw = met.formula_weight # get molecular weight of metabolite in g/mol
        try:
            weight_fraction = (mw/1000)*stoich #the g/gcdw of this metabolite
            biomass_g_g_new.append(weight_fraction)
        except TypeError: #for the met with R in it, just ignore
            print(met.id)

In [38]:
#to find total g/gcdw of all mets, sum up this list
abs(sum(biomass_g_g_new))

0.9990312579196647

In [ ]:
#save&commit
cobra.io.write_sbml_model(model,'../../model/g-thermo.xml')